# Figure 4e

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
sc.settings.set_figure_params(dpi=300)

In [ ]:
data1 = sc.read('/home/jovyan/YS_project/YS/Data_objects/YS_object_14092021/Combined_YS_Fliver_20210929.h5ad')
data2 = sc.read('/home/jovyan/wgeted_files/gastrulation.h5ad')
liver_rest = sc.read('/home/jovyan/mount_farm/nfs/team298/ar32/YS/All_fetal_liver_data/A5_fetal_liver_alladata_updated_annots_210721.h5ad')
liver_adult = sc.read('/home/jovyan/mount_farm/nfs/team298/ar32/YS/adult_liver/data_set1_sharma_not_raw/Liver_raw_20220303.h5ad')

In [ ]:
data2 = data2[data2.obs['corr_concat'].isin(['YS Endoderm_Yolk Sac_gastrulation'])]

data1.obs['celltypes_for_dotplot'] = data1.obs['broad_cell_labels_organ']
data2.obs['celltypes_for_dotplot'] = data2.obs['corr_concat']

adata_list = [data1,data2]
adata = sc.AnnData.concatenate(*adata_list, join='inner', batch_categories=None ,index_unique=None)

cells_to_keep = ['Endoderm_ys','Hepatocyte_fliv','YS Endoderm_Yolk Sac_gastrulation']
adata = adata[adata.obs['celltypes_for_dotplot'].isin(cells_to_keep)]

adata.obs["celltypes_for_dotplot"] = adata.obs["celltypes_for_dotplot"].astype('category')
adata.obs["celltypes_for_dotplot"] = adata.obs["celltypes_for_dotplot"].cat.reorder_categories(['Endoderm_ys','YS Endoderm_Yolk Sac_gastrulation', 'Hepatocyte_fliv'])

In [ ]:
stages= ['CS7','CS10', 'CS11', 'CS22', 'CS23', 'CS14', 'CS18', 'CS15','F61', 'F35', 'F32']
groups = ['CS7','CS10_11', 'CS10_11', 'CS22_23', 'CS22_23', 'CS14_15', 'CS18', 'CS14_15','liv_CS18','liv_CS22_23','liv_CS22_23']
dic = dict(zip(stages,groups))
adata.obs['stage_groups'] = adata.obs['stage'].map(dic)
adata.obs['stage_groups'] = adata.obs['stage_groups'].astype(str)
adata.obs.loc[adata.obs['stage_groups'] == 'nan', 'stage_groups'] = 'Gast_CS7'
adata.obs['stage_groups'] = adata.obs['stage_groups'].astype('category')
adata.obs['stage_groups'] = adata.obs['stage_groups'].cat.reorder_categories(['Gast_CS7','CS10_11','CS14_15','CS18','CS22_23','liv_CS18','liv_CS22_23'])

In [ ]:
liver_rest = liver_rest[~liver_rest.obs['orig.ident'].isin(['F61', 'F35', 'F32'])]

liver_rest.obs['celltypes_for_dotplot'] = liver_rest.obs['cell.labels']
liver_rest = liver_rest[liver_rest.obs['celltypes_for_dotplot'].isin(['Hepatocyte'])]
liver_rest.obs['stage_groups'] = 'fetal_liver'

adata_list = [adata,liver_rest]
adata = sc.AnnData.concatenate(*adata_list, join='inner', batch_categories=None ,index_unique=None)

In [ ]:
liver_adult = liver_adult[liver_adult.obs['NormalvsTumor'].isin(['Normal'])]

liver_adult.obs['celltypes_for_dotplot'] = liver_adult.obs['HCCF1F2_anno']
liver_adult.obs['stage_groups'] = 'adult_liver'

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)

sc.pp.normalize_total(liver_adult)
sc.pp.log1p(liver_adult)
sc.pp.scale(liver_adult, max_value=10)

In [ ]:
liver_adult = liver_adult[liver_adult.obs['celltypes_for_dotplot'].isin(['Hepatocytes'])]

adata_list = [adata,liver_adult]
adata = sc.AnnData.concatenate(*adata_list, join='inner', batch_categories=None ,index_unique=None)

In [ ]:
genes = {
'Common_pathway':['F2','F5','F10'],
    'Extrinsic_pathway':['F3','F7'],
    'Intrinsic_pathway':['F12','F11','F8'],
    'Fibrinogen_and_crosslinking':['FGA','FGB','FGG','F13A1','F13B'],
    'Anticoagulation':['SERPINC1','PROC','PROS1'],
    'Fibrinolysis':['SERPINF2','PLAT','PLAU','SERPINE1','SERPINB2'] # 'SERPINB2' = 'PAI2'
}

adata.obs['stage_groups'] = adata.obs['stage_groups'].astype('category')
adata.obs['stage_groups'] = adata.obs['stage_groups'].cat.reorder_categories(['Gast_CS7','CS10_11','CS14_15','CS18','CS22_23','liv_CS18','liv_CS22_23','fetal_liver','adult_liver'])

sc.pl.dotplot(adata, var_names=genes, groupby='stage_groups', swap_axes=False, figsize=(8,2.5), use_raw=False, save='4e_clotting_actors_over_time_in_human_YS_20220310.pdf')